In [ ]:
import tensorflow as tf
input_shape = (150, 150, 3)
batch_size = 16

In [ ]:
import os 
train_path = os.getcwd() + "/drive/MyDrive/Dataset/flowers/train"
val_path = os.getcwd() + "/drive/MyDrive/Dataset/flowers/val"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   featurewise_center=False,
                                   samplewise_center=False,
                                   featurewise_std_normalization=False,
                                   samplewise_std_normalization=False,
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   vertical_flip=False,
                                   zca_whitening=False
                                   )
val_datagen = ImageDataGenerator(rescale=1. / 255
                                   )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=input_shape[:-1],
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
val_generator = val_datagen.flow_from_directory(val_path,
                                                    target_size=input_shape[:-1],
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
num_class = 1

Found 3237 images belonging to 5 classes.
Found 1080 images belonging to 5 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max',patience=3)

history = model.fit(train_generator,
        steps_per_epoch=len(train_generator),
        epochs=100,
        validation_data=val_generator,
        validation_steps=len(val_generator),
        shuffle=True, 
        verbose = 1,
        callbacks =[callbacks])
len(history.history['val_loss'])  # Only 4 epochs are run

Epoch 1/100
203/203 [==============================] - 894s 4s/step - loss: 1.2669 - accuracy: 0.4520 - val_loss: 0.9873 - val_accuracy: 0.6259
Epoch 2/100
203/203 [==============================] - 136s 669ms/step - loss: 1.0350 - accuracy: 0.5833 - val_loss: 0.9978 - val_accuracy: 0.5972
Epoch 3/100
203/203 [==============================] - 138s 680ms/step - loss: 0.9572 - accuracy: 0.6284 - val_loss: 1.0225 - val_accuracy: 0.6185
Epoch 4/100
203/203 [==============================] - 135s 665ms/step - loss: 0.9082 - accuracy: 0.6420 - val_loss: 0.8508 - val_accuracy: 0.6556
Epoch 5/100
203/203 [==============================] - 135s 664ms/step - loss: 0.8691 - accuracy: 0.6713 - val_loss: 0.7901 - val_accuracy: 0.6972
Epoch 6/100
203/203 [==============================] - 135s 662ms/step - loss: 0.8436 - accuracy: 0.6750 - val_loss: 0.7882 - val_accuracy: 0.7037
Epoch 7/100
203/203 [==============================] - 134s 660ms/step - loss: 0.8257 - accuracy: 0.6973 - val_loss: 0.76

50

In [ ]:
model_json = model.to_json()
with open("model_sequential.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_sequential.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.applications.imagenet_utils import decode_predictions
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  predictions = model.predict_classes(images, batch_size=10)
  if(predictions[0]==0):
    print("Daisy")
  elif(predictions[0]==1):
    print("Dandelion")
  elif(predictions[0]==2):
    print("Rose")
  elif(predictions[0]==3):
    print("Sunflower")
  else:
    print("Tulip")


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()